#Profiling Pecan Consumers

This notebook profiles the consumers in the Pecan St dataset who have HVAC using a thermal regimes model using whole-home consumption and temperature

##Initializations

In [42]:
rm(list = ls())
options(error = recover)
library('segmented')
library('lubridate')
library('zoo')

In [43]:
setwd("~/EnergyAnalytics/batch/pecan/")
source('define_categories_pecan.r')

setwd("~/EnergyAnalytics/utils/")
source('select_data.r')

setwd('~/EnergyAnalytics/thermal_profiles/profiler/')
source('stateProcessorWrapper.r')
source('stateVisualizerWrapper.r')

setwd("~/EnergyAnalytics/batch/pecan/")

In [44]:
DATA_PATH = '~/S3L_server/energy-data/pecan_street/usage-select/'
DUMP_PATH = '~/S3L_server/energy-data/pecan_street/models_new/'
PLOT_PATH = '~/S3L_server/plots/pecan-street-new/'

In [45]:
# load user names
user_names = read.csv('~/S3L_server/energy-data/pecan_street/metadata/user_names_ids.csv')
user_names$X = NULL

# list already processed files
files.input = list.files(path=DUMP_PATH, pattern = '*_decoded*', full.names = T, recursive = T)
already_done  = lapply(files.input, function(x) {
    tmp = tail(strsplit(x, '/')[[1]],n=2)
    res = tmp[1]
    nfo = strsplit(tmp[2], "_")[[1]]
    uid = nfo[1]; nfo = nfo[2]; 
    return(c(uid,res))
})
already_done = data.frame(do.call('rbind', already_done))
if (length(already_done)>0) names(already_done) <- c("ID", "grain")

# list all data files
files    = list.files(path=DATA_PATH, full.names = T, recursive = T)
files_01 = files[grep('01min',files)]
files_15 = files[grep('15min',files)]
files_60 = files[grep('60min',files)]

# extract ID
users_df = data.frame(UID = as.character(sapply(files_60, function(s) strsplit(tail(strsplit(s, '/')[[1]], 1), '\\.')[[1]][1])))
rownames(users_df) = NULL
# users_df['file_01min'] = files_01
users_df['15min'] = files_15
users_df['60min'] = files_60
    
# build data sources dataframe
users_df = merge(user_names, users_df, by.x="ID", by.y="UID")
users_df = melt(users_df, id=c("ID","name"))
names(users_df)[c(3,4)] = c("grain", "file_orig")  
    
# filter out those IDS already done
users_df = subset(users_df, !((ID %in% already_done$ID & grain %in% already_done$grain)))
print(paste("To process:", dim(users_df)[1], "files"))

[1] "To process: 12 files"


In [46]:
head(users_df)

,ID,name,grain,file_orig
116,3368,Forrest,15min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/3368.csv
165,4505,Milton,15min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/4505.csv
171,4670,Ivan,15min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/4670.csv
319,9248,Israel,15min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/9248.csv
335,9647,Ezekiel,15min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/9647.csv
345,9776,Issac,15min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/9776.csv


##Learn thermal regimes models

In [47]:
# function to format data to work with the R thermal profiler code
format_data = function(homeData) {

    # interpolate small gaps
    cat(paste("Initially nrows=",nrow(homeData), '\n'))
    homeData$use = na.spline(homeData$use, maxgap = 12, na.rm=FALSE)
    
    # remove observations with NAs in response
    idx.na = is.na(homeData$use)
    if (sum(idx.na)>0) homeData = homeData[!idx.na,]
    cat(paste("\tRemoved NAs: nrows=",nrow(homeData),"\n"))

    # temperature above reference
    homeData$TemperatureD = homeData$TemperatureF - 65

    # format data as expected by the HMM package
    cur_data = subset(homeData, select = c('date', 'use'))
    names(cur_data)[2] = 'obs'
    cur_data$date = as.character(cur_data$date)
    cur_covar = subset(homeData, select = c('date', 'TemperatureF', 'TemperatureD'))
    cur_covar$date = as.character(cur_covar$date)
    cur_month     = month(cur_data$date)
    cur_covar$TemperatureDWinter = cur_covar$TemperatureD * (cur_month %in% c(0,1,2,3,10,11,12))

    return(list(cur_data, cur_covar))
}


In [48]:
apply_thermal_model = function(cur_data, cur_covar, userName, 
                               dump_path = NULL, 
                               plot_path = NULL) {
  
    # define model learning controls
    controls = list(
        Kmin = 2, Kmax = 6, 
        maxit = 50, 
        nRestarts = 5, 
        tol = 1e-5,
        thresh.R2 = 0.75, 
        thresh.MAPE = 0.25,
        test.periods = 12,
        vis.interval = 3 * 24
    )

    # generate visualization interval; make sure there's data in there
    # TODO: there was an error generated here (indices for subsetting were messed up)
    ok = FALSE
    no.secs    = controls$vis.interval * 3600
    while (!ok) {
        idx_start  = 1
        idx_end    = max(nrow(cur_data)-controls$vis.interval-1, 1)
        start_date = sample(cur_data$date[idx_start:idx_end], 1)
        stop_date  = as.character(as.POSIXct(start_date) + no.secs)
        dat        = subset(cur_data, date >= start_date & date < stop_date)
        if (nrow(na.omit(dat)) > 0) 
          ok = TRUE
    }        

    # learn model
    res = try(stateProcessorWrapper(cur_data, 
                                    cur_covar, 
                                    userName, 
                                    controls = controls,
                                    train.frac = 1.0, 
                                    verbose = F, 
                                    resp.vars = c('(Intercept)', 
                                                  'TemperatureD'), 
                                                  #'TemperatureDWinter'),
                                    dump_path = dump_path))
    if (class(res) == 'try-error') {
        cat(paste('Error in learning model for', userName,'! \n'))
        return(NULL)
    }

    # produce visualizations
    if (!is.null(plot_path)) {
        resv = try(stateVisualizerWrapper(res$decoder, 
                                   res$interpreter, 
                                   plots_path = plot_path, 
                                   interval = c(start_date, stop_date)))
        if (class(resv) == 'try-error') {
            cat('Error in visualizing current user!\n')
        }
    }
    return(NULL)
}


In [49]:
f_wrapper <- function(i) {
    # load data   
    user_id = paste(users_df[i,"ID" ], users_df[i,"name"], sep="_")
    cat(paste('Processing file', users_df[i,'file_orig'], ':', i, '/', nrow(users_df), '\n'))  
    homeData = read.csv(users_df[i,"file_orig"])     
    
    # is there enough data?
    if (is.null(homeData))  {
        cat('Too little data!\n')
        return(NULL)
    }  
    
    # create directory to store models
    dump_path = file.path(DUMP_PATH, paste(user_id, users_df[i,"grain"], sep='/')); 
    dir.create(dump_path, recursive = T)
    # create directory to store plots
    plot_path = file.path(PLOT_PATH, paste(user_id, users_df[i,"name"], sep='/')); 
    dir.create(plot_path, recursive = T)

    # format dataset
    res = format_data(homeData); cur_data = res[[1]]; cur_covar = res[[2]];
    
    # apply model to data
    res = apply_thermal_model(cur_data, cur_covar, user_id, 
                            dump_path = dump_path, 
                            plot_path = plot_path)  
    return(NULL)
}


In [50]:
# library('parallel')
# res = mclapply(1:nrow(users_df), mc.cores = 3, f_wrapper)


In [51]:
setwd("~/EnergyAnalytics/batch/pecan/")
source('define_categories_pecan.r')

setwd("~/EnergyAnalytics/utils/")
source('select_data.r')
    
setwd('~/EnergyAnalytics/thermal_profiles/profiler/')
source('stateProcessorWrapper.r')
source('stateVisualizerWrapper.r')

setwd("~/EnergyAnalytics/batch/pecan/")
#fit = f_wrapper(474)

In [54]:
which(users_df$name=='Lucius' & users_df$grain=='60min')

integer(0)

In [53]:
users_df = users_df[with(users_df, order(ID,grain,name)),]
for (i in 1:nrow(users_df)) {
    cat(paste("*****", i, ":", users_df[i,c("name", "ID")], "*****"))
    f_wrapper(i)
}

***** 1 : 198 ***** ***** 1 : 3368 *****Processing file /Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/3368.csv : 1 / 12 


Warning message:
In dir.create(dump_path, recursive = T): '/Users/adrianalbert/S3L_server/energy-data/pecan_street/models_new//3368_Forrest/15min' already existsWarning message:
In dir.create(plot_path, recursive = T): '/Users/adrianalbert/S3L_server/plots/pecan-street-new//3368_Forrest/Forrest' already exists

Initially nrows= 18908 
	Removed NAs: nrows= 17852 
*** Initializing DataFormatter (3368_Forrest) ***
   user  system elapsed 
  0.001   0.001   0.001 
*** Adding covariates data for DataFormatter3368_Forrest ***
   user  system elapsed 
  0.006   0.001   0.009 
*** Initializing StateDecoder (3368_Forrest) ***
   user  system elapsed 
  0.007   0.002   0.008 
***** HMM Cross-Validation K = 2 ******
iteration 0 logLik: -8996.197 
Bad HMM fit; re-estimating...
iteration 0 logLik: -9843.166 
iteration 5 logLik: 717.1247 
Bad HMM fit; re-estimating...
iteration 0 logLik: -9634.113 
Bad HMM fit; re-estimating...
iteration 0 logLik: -9533.372 
Bad HMM fit; re-estimating...
iteration 0 logLik: -9723.359 
Bad HMM fit; re-estimating...
iteration 0 logLik: -9831.699 
Bad HMM fit; re-estimating...
[1] "MAPE=NA; R2=NA"
***** HMM Cross-Validation K = 3 ******
iteration 0 logLik: -9818.8 
iteration 5 logLik: 1784.186 
Bad HMM fit; re-estimating...
iteration 0 logLik: -9418.663 
iteration 5 logLik: 3

Warning message:
In dir.create(dump_path, recursive = T): '/Users/adrianalbert/S3L_server/energy-data/pecan_street/models_new//3368_Forrest/60min' already existsWarning message:
In dir.create(plot_path, recursive = T): '/Users/adrianalbert/S3L_server/plots/pecan-street-new//3368_Forrest/Forrest' already exists

Initially nrows= 4727 
	Removed NAs: nrows= 4463 
*** Initializing DataFormatter (3368_Forrest) ***
   user  system elapsed 
      0       0       0 
*** Adding covariates data for DataFormatter3368_Forrest ***
   user  system elapsed 
  0.001   0.001   0.002 
*** Initializing StateDecoder (3368_Forrest) ***
   user  system elapsed 
  0.002   0.000   0.002 
***** HMM Cross-Validation K = 2 ******
iteration 0 logLik: -2107.114 
iteration 5 logLik: -76.15341 
Bad HMM fit; re-estimating...
iteration 0 logLik: -1886.891 
iteration 5 logLik: 531.8979 
Bad HMM fit; re-estimating...
iteration 0 logLik: -2107.481 
iteration 5 logLik: -203.8127 
Bad HMM fit; re-estimating...
iteration 0 logLik: -2107.406 
iteration 5 logLik: -177.4545 
Bad HMM fit; re-estimating...
iteration 0 logLik: -2095.9 
iteration 5 logLik: 172.45 
Bad HMM fit; re-estimating...
iteration 0 logLik: -2092.521 
iteration 5 logLik: 190.025 
Bad HMM fit; re-estimating...
[1] "MAPE=NA; R2=NA"
***** HMM Cross-Validation K = 3 **

...

Initially nrows= 4308 
	Removed NAs: nrows= 4308 
*** Initializing DataFormatter (9647_Ezekiel) ***
   user  system elapsed 
  0.000   0.000   0.001 
*** Adding covariates data for DataFormatter9647_Ezekiel ***
   user  system elapsed 
  0.001   0.000   0.001 
*** Initializing StateDecoder (9647_Ezekiel) ***
   user  system elapsed 
  0.001   0.000   0.001 
***** HMM Cross-Validation K = 2 ******
iteration 0 logLik: -66.59136 
Bad HMM fit; re-estimating...
iteration 0 logLik: 543.4801 
Bad HMM fit; re-estimating...
iteration 0 logLik: -1134.025 
Bad HMM fit; re-estimating...
iteration 0 logLik: -371.9522 
Bad HMM fit; re-estimating...
iteration 0 logLik: -722.3964 
Bad HMM fit; re-estimating...
iteration 0 logLik: -1131.49 
Bad HMM fit; re-estimating...
[1] "MAPE=NA; R2=NA"
***** HMM Cross-Validation K = 3 ******
iteration 0 logLik: 416.3265 
Bad HMM fit; re-estimating...
iteration 0 logLik: 4003.097 
Bad HMM fit; re-estimating...
iteration 0 logLik: 337.7238 
Bad HMM fit; re-estimatin

Warning message:
In dir.create(dump_path, recursive = T): '/Users/adrianalbert/S3L_server/energy-data/pecan_street/models_new//9776_Issac/15min' already existsWarning message:
In dir.create(plot_path, recursive = T): '/Users/adrianalbert/S3L_server/plots/pecan-street-new//9776_Issac/Issac' already exists

Initially nrows= 17232 
	Removed NAs: nrows= 17202 
*** Initializing DataFormatter (9776_Issac) ***
   user  system elapsed 
  0.002   0.000   0.002 
*** Adding covariates data for DataFormatter9776_Issac ***
   user  system elapsed 
  0.011   0.001   0.012 
*** Initializing StateDecoder (9776_Issac) ***
   user  system elapsed 
  0.003   0.000   0.002 
***** HMM Cross-Validation K = 2 ******
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
[1] "MAPE=NA; R2=NA"
***** HMM Cross-Validation K = 3 ******
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
[1] "MAPE=NA; R2=NA"
***** HMM Cross-Validation K = 4 ******
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Ba

Warning message:
In dir.create(dump_path, recursive = T): '/Users/adrianalbert/S3L_server/energy-data/pecan_street/models_new//9776_Issac/60min' already existsWarning message:
In dir.create(plot_path, recursive = T): '/Users/adrianalbert/S3L_server/plots/pecan-street-new//9776_Issac/Issac' already exists

Initially nrows= 4308 
	Removed NAs: nrows= 4308 
*** Initializing DataFormatter (9776_Issac) ***
   user  system elapsed 
  0.001   0.000   0.001 
*** Adding covariates data for DataFormatter9776_Issac ***
   user  system elapsed 
  0.001   0.000   0.001 
*** Initializing StateDecoder (9776_Issac) ***
   user  system elapsed 
  0.001   0.000   0.001 
***** HMM Cross-Validation K = 2 ******
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
[1] "MAPE=NA; R2=NA"
***** HMM Cross-Validation K = 3 ******
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
[1] "MAPE=NA; R2=NA"
***** HMM Cross-Validation K = 4 ******
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
Bad 